<a href="https://colab.research.google.com/github/DGKobelev/Home/blob/main/intern_3/unilit_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


Преобразование в формат csv

In [ ]:
import pandas as pd

# Загрузка файла Excel
combined_df = pd.read_excel('/content/drive/MyDrive/Unilit/Копия Обращения.xlsx', engine='openpyxl')

# Сохранение в формат CSV
csv_file_path = '/content/drive/MyDrive/Unilit/appeals_file.csv'
combined_df.to_csv(csv_file_path, index=False, encoding='utf-8')

print(f"Файл успешно сохранен в формате CSV: {csv_file_path}")


Файл успешно сохранен в формате CSV: /content/drive/MyDrive/Unilit/appeals_file.csv


Перевод текстов к нижнему регистру и удаление лишних пробелов и символов.

In [ ]:
!pip install pymorphy2

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 84.0 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=aee9e461db7001d24640edc0128a01168e9be49bcf5c9b8aaa525a96cff55b53
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
import pandas as pd
import pymorphy2
import re
import nltk
from nltk.corpus import stopwords

# Загрузка стоп-слов (требуется одноразовая загрузка)
nltk.download('stopwords')

# Инициализация анализатора pymorphy2
ma = pymorphy2.MorphAnalyzer()

# Определение стоп-слов для русского языка
stop_words = set(stopwords.words('russian'))

# Загрузка данных, игнорируя первый ряд как заголовок
file_path = '/content/drive/MyDrive/Unilit/appeals_file.csv'
combined_df = pd.read_csv(file_path, header=1)  # Загружаем, начиная со второго ряда

# Переименование столбцов
combined_df.columns = ['№', 'Дата обращения', 'Суть обращения', 'Характер обращ', 'Комментарии инженера']

# Функция для очистки и нормализации текста
def clean_text(text):
    if not isinstance(text, str):  # Проверка, что text является строкой
        return ""
    # Удаление ненужных символов
    text = re.sub(r'[^\w\s]', '', text)
    text = text.replace("\\", " ").replace(u"╚", " ").replace(u"╩", " ")
    text = text.lower()
    text = re.sub('-\s*\r\n|\r\n', '', text)  # удаление переносов строк
    text = " ".join(ma.parse(word)[0].normal_form for word in text.split())
    text = re.sub('\s+', ' ', text)  # замена нескольких пробелов одним
    # Удаление стоп-слов
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

# Применение функции очистки и нормализации к столбцам 'Суть обращения' и 'Комментарии инженера'
combined_df['appeals_reduct'] = combined_df['Суть обращения'].apply(clean_text)
combined_df['engineer_comments_reduct'] = combined_df['Комментарии инженера'].apply(clean_text)

# Сохранение очищенного DataFrame в формате CSV
output_csv_path = '/content/drive/MyDrive/Unilit/clean_appeals_file.csv'
combined_df.to_csv(output_csv_path, index=False)

print("Данные успешно очищены и сохранены.")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Данные успешно очищены и сохранены.


In [ ]:
import pandas as pd
import pymorphy2
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Инициализация анализатора pymorphy2
ma = pymorphy2.MorphAnalyzer()

# Загрузка данных
file_path = '/content/drive/MyDrive/Unilit/clean_appeals_file.csv'
combined_df = pd.read_csv(file_path)

# Замена NaN на пустые строки в столбцах 'appeals_reduct' и 'engineer_comments_reduct'
combined_df['appeals_reduct'] = combined_df['appeals_reduct'].fillna('')
combined_df['engineer_comments_reduct'] = combined_df['engineer_comments_reduct'].fillna('')

# Подготовка признаков с помощью TF-IDF на основе столбцов 'Суть обращения' и 'Комментарии инженера'
tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(combined_df['appeals_reduct'] + " " + combined_df['engineer_comments_reduct'])

# Целевая переменная - классы из столбца 'Характер обращ'
y = combined_df['Характер обращ'].fillna('')  # Замена NaN на пустые строки в целевой переменной

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Проверка и удаление строк с NaN в обучающих данных
# В случае, если после разбиения всё ещё остались NaN
X_train = X_train[~pd.isnull(y_train)]
y_train = y_train[~pd.isnull(y_train)]

# Инициализация модели логистической регрессии
model = LogisticRegression(max_iter=1000)

# Обучение модели
model.fit(X_train, y_train)

# Предсказания на тестовых данных
y_pred = model.predict(X_test)

# Оценка точности модели
accuracy = accuracy_score(y_test, y_pred)
print(f"Точность модели: {accuracy:.4f}")

# Полный отчёт по классификации (Precision, Recall, F1-score)
print("\nОтчёт по классификации:")
print(classification_report(y_test, y_pred))

# Сохранение очищенного DataFrame в формате CSV (если нужно)
output_csv_path = '/content/drive/MyDrive/Unilit/final_appeals_file.csv'
combined_df.to_csv(output_csv_path, index=False)

print("Данные успешно подготовлены и модель оценена.")


Точность модели: 0.7812

Отчёт по классификации:
               precision    recall  f1-score   support

Неисправность       0.78      1.00      0.88        25
   Расписание       0.00      0.00      0.00         1
        Связь       0.00      0.00      0.00         6

     accuracy                           0.78        32
    macro avg       0.26      0.33      0.29        32
 weighted avg       0.61      0.78      0.69        32

Данные успешно подготовлены и модель оценена.


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


судя по отчёту, модель показывает хорошие результаты для класса "Неисправность", но не справляется с другими классами ("Расписание" и "Связь")

In [ ]:
import pandas as pd
import pymorphy2
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Инициализация анализатора pymorphy2
ma = pymorphy2.MorphAnalyzer()

# Загрузка данных
file_path = '/content/drive/MyDrive/Unilit/clean_appeals_file.csv'
combined_df = pd.read_csv(file_path)

# Замена NaN на пустые строки в столбцах 'appeals_reduct' и 'engineer_comments_reduct'
combined_df['appeals_reduct'] = combined_df['appeals_reduct'].fillna('')
combined_df['engineer_comments_reduct'] = combined_df['engineer_comments_reduct'].fillna('')

# Подготовка признаков с помощью TF-IDF на основе столбцов 'Суть обращения' и 'Комментарии инженера'
tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(combined_df['appeals_reduct'] + " " + combined_df['engineer_comments_reduct'])

# Целевая переменная - классы из столбца 'Характер обращ'
y = combined_df['Характер обращ'].fillna('')  # Замена NaN на пустые строки в целевой переменной

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Проверка и удаление строк с NaN в обучающих данных
# В случае, если после разбиения всё ещё остались NaN
X_train = X_train[~pd.isnull(y_train)]
y_train = y_train[~pd.isnull(y_train)]

# Инициализация модели логистической регрессии с учетом баланса классов
model = LogisticRegression(max_iter=1000, class_weight='balanced')

# Обучение модели
model.fit(X_train, y_train)

# Предсказания на тестовых данных
y_pred = model.predict(X_test)

# Оценка точности модели
accuracy = accuracy_score(y_test, y_pred)
print(f"Точность модели: {accuracy:.4f}")

# Полный отчёт по классификации (Precision, Recall, F1-score)
print("\nОтчёт по классификации:")
print(classification_report(y_test, y_pred))


Точность модели: 0.5938

Отчёт по классификации:
               precision    recall  f1-score   support

                    0.00      0.00      0.00         0
Неисправность       0.94      0.64      0.76        25
   Расписание       0.00      0.00      0.00         1
        Связь       1.00      0.50      0.67         6

     accuracy                           0.59        32
    macro avg       0.49      0.29      0.36        32
 weighted avg       0.92      0.59      0.72        32



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import pandas as pd
import pymorphy2
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Инициализация анализатора pymorphy2
ma = pymorphy2.MorphAnalyzer()

# Загрузка данных
file_path = '/content/drive/MyDrive/Unilit/clean_appeals_file.csv'
combined_df = pd.read_csv(file_path)

# Удаление строк с пустыми значениями в классе 'Характер обращ'
combined_df = combined_df[combined_df['Характер обращ'].notna() & (combined_df['Характер обращ'] != '')]

# Замена NaN на пустые строки в столбцах 'appeals_reduct' и 'engineer_comments_reduct'
combined_df['appeals_reduct'] = combined_df['appeals_reduct'].fillna('')
combined_df['engineer_comments_reduct'] = combined_df['engineer_comments_reduct'].fillna('')

# Подготовка признаков с помощью TF-IDF на основе столбцов 'Суть обращения' и 'Комментарии инженера'
tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(combined_df['appeals_reduct'] + " " + combined_df['engineer_comments_reduct'])

# Целевая переменная - классы из столбца 'Характер обращ'
y = combined_df['Характер обращ'].fillna('')  # Замена NaN на пустые строки в целевой переменной

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Удаление строк с NaN в обучающих и тестовых данных
X_train = X_train[~pd.isnull(y_train)]
y_train = y_train[~pd.isnull(y_train)]

X_test = X_test[~pd.isnull(y_test)]
y_test = y_test[~pd.isnull(y_test)]

# Инициализация модели Random Forest
model = RandomForestClassifier(class_weight='balanced', random_state=42)

# Обучение модели
model.fit(X_train, y_train)

# Предсказания на тестовых данных
y_pred = model.predict(X_test)

# Оценка точности модели
accuracy = accuracy_score(y_test, y_pred)
print(f"Точность модели: {accuracy:.4f}")

# Полный отчёт по классификации (Precision, Recall, F1-score)
print("\nОтчёт по классификации:")
print(classification_report(y_test, y_pred))


Точность модели: 0.7812

Отчёт по классификации:
               precision    recall  f1-score   support

Неисправность       0.80      0.96      0.87        25
        Связь       0.50      0.14      0.22         7

     accuracy                           0.78        32
    macro avg       0.65      0.55      0.55        32
 weighted avg       0.73      0.78      0.73        32



1. **Загрузка данных**:
   данные из CSV-файла, содержащие текстовые данные о "Суть обращения" и "Комментарии инженера", а также классы обращений в столбце "Характер обращ".

2. **Предобработка данных**:
   - **Очистка текста**: Все текстовые данные очищаются от пропущенных значений (NaN) в столбцах `'appeals_reduct'` и `'engineer_comments_reduct'`. Это важно для того, чтобы текст был чистым и готовым для векторизации.
   - **Удаление пустых строк в столбце классов**: удаляем строки, где в столбце `'Характер обращ'` есть пустые значения, чтобы избежать проблем при обучении модели.

3. **Применение TF-IDF**:
   - **TF-IDF векторизация**: Ты применяем `TfidfVectorizer`, который преобразует текст в числовые признаки (векторы). Это необходимо, чтобы алгоритмы машинного обучения могли работать с текстом.
   - **Что делает TF-IDF**:
     - **TF (Term Frequency)**: рассчитывает частоту появления слова в документе.
     - **IDF (Inverse Document Frequency)**: снижает вес слов, которые часто встречаются в наборе данных, и увеличивает вес редких слов.
   - В результате каждое обращение и комментарий превращаются в набор чисел, которые описывают важность слов.

4. **Разделение данных на обучающую и тестовую выборки**:
   разделяем данные на обучающие (80%) и тестовые (20%) с помощью `train_test_split`.

5. **Инициализация модели Random Forest**:
   - **Random Forest** — это ансамблевый алгоритм, который использует несколько деревьев решений для классификации. Каждое дерево строится на случайной подвыборке признаков, что делает модель более устойчивой и менее склонной к переобучению.
   - используем параметр `class_weight='balanced'`, чтобы модель учитывала дисбаланс классов (когда один класс сильно преобладает над другими). Это помогает сбалансировать вес для каждого класса.

6. **Обучение модели**:
   Модель Random Forest обучается на векторизованных текстах и классах обращений. Она строит несколько деревьев решений и использует их, чтобы предсказать класс на основе данных.

7. **Предсказание на тестовых данных**:
   После обучения модель предсказывает классы для тестовой выборки (20% данных, которые не использовались для обучения). Это помогает проверить, насколько хорошо модель может классифицировать новые, ранее невидимые данные.

8. **Оценка модели**:
   - **Точность модели** (`accuracy`) — это доля правильных предсказаний от общего числа примеров.
   - **Отчёт по классификации** (`classification_report`) показывает более детальные метрики для каждого класса:
     - **Precision** — точность: доля правильных предсказаний данного класса среди всех предсказанных как этот класс.
     - **Recall** — полнота: доля правильно предсказанных примеров данного класса среди всех реальных примеров этого класса.
     - **F1-score** — гармоническое среднее между Precision и Recall.
     - **Support** — количество примеров в тестовой выборке для каждого класса.

### Результаты:
- Модель достигла точности **0.7812**, что означает, что 78.12% предсказаний были верными.
- **Класс "Неисправность"**: имеет высокую точность и полноту, так как этот класс доминирует в данных (25 примеров из 32). Модель хорошо его предсказывает.
- **Класс "Связь"**: здесь модель показывает низкие значения Recall (0.14), так как у неё мало данных для этого класса, что мешает ей правильно идентифицировать такие случаи.

### Основные выводы:
- Модель хорошо обучилась на большом классе "Неисправность", но плохо справляется с меньшим классом "Связь", что типично для несбалансированных данных.
- Использование **class_weight='balanced'** помогло немного улучшить результаты для меньших классов.